<a href="https://colab.research.google.com/github/Tejas163/Deep_Learning/blob/main/minilib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Parsing MNIST

In [3]:
# prompt: def parse_mnist(image_filename, label_filename):
#     """ Read an images and labels file in MNIST format.  See this page:
#     http://yann.lecun.com/exdb/mnist/ for a description of the file format.
#     Args:
#         image_filename (str): name of gzipped images file in MNIST format
#         label_filename (str): name of gzipped labels file in MNIST format
#     Returns:
#         Tuple (X,y):
#             X (numpy.ndarray[np.float32]): 2D numpy array containing the loaded
#                 data.  The dimensionality of the data should be
#                 (num_examples x input_dim) where 'input_dim' is the full
#                 dimension of the data, e.g., since MNIST images are 28x28, it
#                 will be 784.  Values should be of type np.float32, and the data
#                 should be normalized to have a minimum value of 0.0 and a
#                 maximum value of 1.0 (i.e., scale original values of 0 to 0.0
#                 and 255 to 1.0).
#             y (numpy.ndarray[dtype=np.uint8]): 1D numpy array containing the
#                 labels of the examples.  Values should be of type np.uint8 and
#                 for MNIST will contain the values 0-9.
#     """
#  We'd recommend you use the struct module in python (along with the gzip module and of course numpy itself), in order to implement this function

import gzip
import numpy as np
import struct

def parse_mnist(image_filename, label_filename):
    """ Read an images and labels file in MNIST format.  See this page:
    http://yann.lecun.com/exdb/mnist/ for a description of the file format.
    Args:
        image_filename (str): name of gzipped images file in MNIST format
        label_filename (str): name of gzipped labels file in MNIST format
    Returns:
        Tuple (X,y):
            X (numpy.ndarray[np.float32]): 2D numpy array containing the loaded
                data.  The dimensionality of the data should be
                (num_examples x input_dim) where 'input_dim' is the full
                dimension of the data, e.g., since MNIST images are 28x28, it
                will be 784.  Values should be of type np.float32, and the data
                should be normalized to have a minimum value of 0.0 and a
                maximum value of 1.0 (i.e., scale original values of 0 to 0.0
                and 255 to 1.0).
            y (numpy.ndarray[dtype=np.uint8]): 1D numpy array containing the
                labels of the examples.  Values should be of type np.uint8 and
                for MNIST will contain the values 0-9.
    """
    with gzip.open(image_filename, 'rb') as f:
        magic, num_images, rows, cols = struct.unpack(">IIII", f.read(16))
        images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num_images, rows * cols)
        images = images.astype(np.float32) / 255.0

    with gzip.open(label_filename, 'rb') as f:
        magic, num_labels = struct.unpack(">II", f.read(8))
        labels = np.frombuffer(f.read(), dtype=np.uint8)

    return images, labels

In [4]:
images, labels = parse_mnist('/content/t10k-images-idx3-ubyte.gz', '/content/t10k-labels-idx1-ubyte.gz')
print("Images shape:", images.shape)  # Should be (60000, 784) for the training set
print("Labels shape:", labels.shape)  # Should be (60000,)

Images shape: (10000, 784)
Labels shape: (10000,)


In [ ]:
# prompt: there are five fields within the NDArray class that you'll need to be familiar with (note that the real class member these all these fields is preceded by an underscore, e.g., _handle, _strides, etc, some of which are then exposed as a public property ... for all your code it's fine to use the internal, underscored version).
# device - A object of type BackendDevice, which is a simple wrapper that contains a link to the underlying device backend (e.g., CPU or CUDA).
# handle - A class objected that stores the underlying memory of the array. This is allocated as a class of type device.Array(), though this allocation all happens in the provided code (specifically the NDArray.make function), and you don't need to worry about calling it yourself.
# shape - A tuple specifying the size of each dimension in the array.
# strides - A tuple specifying the strides of each dimension in the array.
# offset - An integer indicating where in the underlying device.Array memory the array actually starts (it's convenient to store this so we can more easily manage pointing back to existing memory, without having to track allocations).
# Of particular importance for many of your Python implementations will be the NDArray.make call:
# def make(shape, strides=None, device=None, handle=None, offset=0):
# which creates a new NDArray with the given shape, strides, device, handle, and offset. If handle is not specified (i.e., no pre-existing memory is referenced), then the call will allocate the needed memory, but if handle is specified then no new memory is allocated, but the new NDArray points the same memory as the old one

class BackendDevice:
    def __init__(self):
        pass  # Replace with actual device initialization if needed


class NDArray:
    def __init__(self, device, handle, shape, strides, offset):
        self._device = device
        self._handle = handle
        self._shape = shape
        self._strides = strides
        self._offset = offset

    @classmethod
    def make(cls, shape, strides=None, device=None, handle=None, offset=0):
        if handle is None:
            # Allocate new memory if handle is not provided.
            # Replace with actual memory allocation based on device and shape
            # For demonstration, using a placeholder
            handle = object()

        if device is None:
            device = BackendDevice()

        if strides is None:
            # Calculate default strides if not given
            strides = tuple([1] * len(shape))

        return cls(device, handle, shape, strides, offset)

    def __repr__(self):
        return f"NDArray(shape={self._shape}, strides={self._strides}, offset={self._offset})"

    def reshape(self, new_shape):
      curr_size=1
      for dim in self._shape:
        curr_size*=dim

      new_size=1
      for dim in new_shape:
        new_size*=dim

      if curr_size!=new_size:
          raise ValueError("Product of current shape is not equal to the product of the new shape.")

      return NDArray(self._device, self._handle, new_shape, None, self._offset)  # strides will be recalculated if needed

    def permute(self, new_axes):
        new_shape = tuple(self._shape[i] for i in new_axes)
        new_strides = tuple(self._strides[i] for i in new_axes)

        return NDArray(self._device, self._handle, new_shape, new_strides, self._offset)


# Example usage
array = NDArray.make((2, 3))
print(array)


# Example usage
array = NDArray.make((2, 3))
print(array)

reshaped_array = array.reshape((3,2))
print(reshaped_array)

premute_array = array.permute((1,0))
print(premute_array)

# Creating an array referencing existing memory (demonstration)
existing_array = NDArray.make((2, 3))
new_array = NDArray.make((2, 3), handle=existing_array._handle)
new_array

NDArray(shape=(2, 3), strides=(1, 1), offset=0)
NDArray(shape=(2, 3), strides=(1, 1), offset=0)
NDArray(shape=(3, 2), strides=None, offset=0)
NDArray(shape=(3, 2), strides=(1, 1), offset=0)


NDArray(shape=(2, 3), strides=(1, 1), offset=0)

Part1

In [ ]:
# prompt: def reshape(self, new_shape):
#         """
#         Reshape the matrix without copying memory.  This will return a matrix
#         that corresponds to a reshaped array but points to the same memory as
#         the original array.
#         Raises:
#             ValueError if product of current shape is not equal to the product
#             of the new shape, or if the matrix is not compact.
#         Args:
#             new_shape (tuple): new shape of the array
#         Returns:
#             NDArray : reshaped array; this will point to thep
#         """
def reshape(self, new_shape):
        """
        Reshape the matrix without copying memory.  This will return a matrix
        that corresponds to a reshaped array but points to the same memory as
        the original array.
        Raises:
            ValueError if product of current shape is not equal to the product
            of the new shape, or if the matrix is not compact.
        Args:
            new_shape (tuple): new shape of the array
        Returns:
            NDArray : reshaped array; this will point to thep
        """
        current_size = 1
        for dim in self._shape:
            current_size *= dim

        new_size = 1
        for dim in new_shape:
            new_size *= dim

        if current_size != new_size:
            raise ValueError("Product of current shape is not equal to the product of the new shape.")

        #  Add check for compact matrix if needed
        #  if not self.is_compact():
        #    raise ValueError("The matrix is not compact.")

        return NDArray(self._device, self._handle, new_shape, None, self._offset)  # strides will be recalculated if needed


NDArray(shape=(2, 3), strides=(1, 1), offset=0)


AttributeError: 'NDArray' object has no attribute 'reshape'

Permute method

In [ ]:
# prompt: def permute(self, new_axes):
#         """
#         Permute order of the dimensions.  new_axes describes a permuation of the
#         existing axes, so e.g.:
#           - If we have an array with dimension "BHWC" then .permute((0,3,1,2))
#             would convert this to "BCHW" order.
#           - For a 2D array, .permute((1,0)) would transpose the array.
#         Like reshape, this operation should not copy memory, but achieves the
#         permuting by just adjusting the shape/strides of the array.  That is,
#         it returns a new array that has the dimensions permuted as desired, but
#         which points to the same memroy as the original array.
#         Args:
#             new_axes (tuple): permuation order of the dimensions
#         Returns:
#             NDarray : new NDArray object with permuted dimensions, pointing
#             to the same memory as the original NDArray (i.e., just shape and
#             strides changed).
#         """

def permute(self, new_axes):
        """
        Permute order of the dimensions.  new_axes describes a permuation of the
        existing axes, so e.g.:
          - If we have an array with dimension "BHWC" then .permute((0,3,1,2))
            would convert this to "BCHW" order.
          - For a 2D array, .permute((1,0)) would transpose the array.
        Like reshape, this operation should not copy memory, but achieves the
        permuting by just adjusting the shape/strides of the array.  That is,
        it returns a new array that has the dimensions permuted as desired, but
        which points to the same memroy as the original array.
        Args:
            new_axes (tuple): permuation order of the dimensions
        Returns:
            NDarray : new NDArray object with permuted dimensions, pointing
            to the same memory as the original NDArray (i.e., just shape and
            strides changed).
        """

        new_shape = tuple(self._shape[i] for i in new_axes)
        new_strides = tuple(self._strides[i] for i in new_axes)

        return NDArray(self._device, self._handle, new_shape, new_strides, self._offset)